In [ ]:
import numpy as np
import pandas as pd

/kaggle/input/a3-dataset/2024-07-23-washington-post-police-shootings-export.csv
/kaggle/input/a3-dataset/sc-est2023-alldata6.csv


In [ ]:
df_police_shootings = pd.read_csv('../datasets/2024-07-23-washington-post-police-shootings-export.csv')
df_demographic = pd.read_csv('../datasets/sc-est2023-alldata6.csv')

In [15]:
# brief view of the police shootings dataset

df_police_shootings.head()

,date,name,age,gender,armed,race,city,state,flee,body_camera,signs_of_mental_illness,police_departments_involved
0,2015-01-02,Lewis Lee Lembke,47.0,male,gun,White,Aloha,OR,not,False,False,"Washington County Sheriff's Office, OR"
1,2015-01-02,Tim Elliot,53.0,male,gun,Asian,Shelton,WA,not,False,True,"Mason County Sheriff's Office, WA"
2,2015-01-03,John Paul Quintero,23.0,male,unarmed,Hispanic,Wichita,KS,not,False,False,"Wichita Police Department, KS"
3,2015-01-04,Kenneth Joe Brown,18.0,male,gun,White,Guthrie,OK,not,False,False,"Oklahoma Highway Patrol, OK"
4,2015-01-04,Michael Rodriguez,39.0,male,other,Hispanic,Evans,CO,not,False,False,"Evans Police Department, CO"


In [16]:
# brief view of the demographic dataset

df_demographic.head()

,SUMLEV,REGION,DIVISION,STATE,NAME,SEX,ORIGIN,RACE,AGE,ESTIMATESBASE2020,POPESTIMATE2020,POPESTIMATE2021,POPESTIMATE2022,POPESTIMATE2023
0,40,3,6,1,Alabama,0,0,1,0,35241,35185,35607,36761,36974
1,40,3,6,1,Alabama,0,0,1,1,36458,36103,35613,36004,37241
2,40,3,6,1,Alabama,0,0,1,2,37746,37517,36408,36106,36542
3,40,3,6,1,Alabama,0,0,1,3,38696,38373,37962,36867,36766
4,40,3,6,1,Alabama,0,0,1,4,39654,39614,38780,38388,37489


In [17]:
# in the original police shootings dataset, Hispanic origin was counted as a separate race, 
# so we will merge Hispanic origin (marked with ORIGIN = 2) into race as a new race, leave
# be those with non-Hispanic origin (marked with ORIGIN = 1), and drop rows of total measure
# (marked with ORIGIN = 0)

df_demographic.loc[df_demographic['ORIGIN'] == 2, 'RACE'] = 7
df_demographic['RACE'].unique()

array([1, 2, 3, 4, 5, 6, 7])

In [18]:
# we will drop the name column from the shootings dataset, since it is functionally irrelevant and
# we cannot perform any analysis on it. we can also drop the rows where ORIGIN is 0 (since the total is
# now incorporated in the races total) and drop the ORIGIN column

df_police_shootings = df_police_shootings.drop('name', axis=1)
df_demographic = df_demographic.drop(df_demographic[df_demographic['ORIGIN'] == 0].index)
df_demographic = df_demographic.drop('ORIGIN', axis=1)

In [19]:
df_police_shootings.columns, df_demographic.columns

(Index(['date', 'age', 'gender', 'armed', 'race', 'city', 'state', 'flee',
        'body_camera', 'signs_of_mental_illness',
        'police_departments_involved'],
       dtype='object'),
 Index(['SUMLEV', 'REGION', 'DIVISION', 'STATE', 'NAME', 'SEX', 'RACE', 'AGE',
        'ESTIMATESBASE2020', 'POPESTIMATE2020', 'POPESTIMATE2021',
        'POPESTIMATE2022', 'POPESTIMATE2023'],
       dtype='object'))

In [21]:
# now there will be multiple rows that have race 7 (since, they are different races with Hispanic origin). 
# we can just merge those rows by adding their sum, provided they have the same statistics otherwise. since theoretically,
# the rows with the same value of SUMLEV, REGION, DIVISION, STATE, NAME, SEX, AGE, RACE will all be Hispanic (since the 
# others have to have some distinction) we can group by those and sum the rows.

df_demographic = df_demographic.groupby(
    ['SUMLEV', 'REGION', 'DIVISION', 'STATE', 'NAME', 'SEX', 'AGE', 'RACE']
).sum().reset_index()

In [22]:
# here, we can see now that we have only one row for each race

df_demographic.head(20)

,SUMLEV,REGION,DIVISION,STATE,NAME,SEX,AGE,RACE,ESTIMATESBASE2020,POPESTIMATE2020,POPESTIMATE2021,POPESTIMATE2022,POPESTIMATE2023
0,40,1,1,9,Connecticut,0,0,1,16745,16609,15531,16191,15805
1,40,1,1,9,Connecticut,0,0,2,4226,4162,4189,4435,4365
2,40,1,1,9,Connecticut,0,0,3,67,61,56,61,59
3,40,1,1,9,Connecticut,0,0,4,1994,1915,1592,1649,1619
4,40,1,1,9,Connecticut,0,0,5,19,19,35,27,27
5,40,1,1,9,Connecticut,0,0,6,1432,1450,1495,1542,1500
6,40,1,1,9,Connecticut,0,0,7,10007,10240,10959,11684,11560
7,40,1,1,9,Connecticut,0,1,1,17291,17112,17227,15874,16584
8,40,1,1,9,Connecticut,0,1,2,4148,4199,4249,4357,4593
9,40,1,1,9,Connecticut,0,1,3,76,74,60,61,61


In [25]:
# in the police shootings dataset, there are semicolon-separated police departments in case one or more PDs are 
# involved in the shooting case. to handle this, several (nullable) columns are made, one with each PD's name,
# by splitting the initial PDs list

df_police_shootings['police_departments_involved'] = df_police_shootings['police_departments_involved'].str.split(';')

In [32]:
df_police_shootings['police_departments_involved']

0                [Washington County Sheriff's Office, OR]
1                     [Mason County Sheriff's Office, WA]
2                         [Wichita Police Department, KS]
3                           [Oklahoma Highway Patrol, OK]
4                           [Evans Police Department, CO]
                              ...                        
9888                       [Easley Police Department, SC]
9889             [Pierce County Sheriff's Department, WA]
9890    [Alaska Wildlife Troopers, AK, Juneau Police D...
9891                     [Beaumont Police Department, TX]
9892                                  [Marion Police, IN]
Name: police_departments_involved, Length: 9893, dtype: object

In [ ]:
df_demographic.to_csv('../datasets/demographic_cleaned.csv')
df_police_shootings.to_csv('../datasets/police_shootings_cleaned.csv')